# Ring Model with LOCM Perturbation Kernel


In [ ]:
import os
import numpy as np
import scipy.stats as st
import matplotlib.pyplot as plt

from abcsmc.abcsmc import abcsmc
from abcsmc.plotting import plot_results
from abcsmc.models import RingModel
from abcsmc.priors import UniformPrior


In [ ]:
imgdir = "../out/images/nb_ring_locm"
os.makedirs(imgdir, exist_ok=True)

### Generate observed data $X_0$


In [ ]:
# True model parameters
theta1 = 3
theta2 = 4
var_const = 1

# True model and generated "observed" data
model = RingModel(theta1, theta2, var_const)
data = model.generate_data(1000)

# Parameter indices and names used for plotting
pidx1 = 0
pidx2 = 1
pname1 = "$\\theta_1$"
pname2 = "$\\theta_2$"

plt.hist(data, density=True)
plt.title("Observed Data")
plt.xlabel('$x$');
plt.ylabel("density");

### Define priors


In [ ]:
# Priors
prior_theta1 = UniformPrior(-20, 20)
prior_theta2 = UniformPrior(-20, 20)

prior_list = [prior_theta1, prior_theta2]

### Compute summary statistics of the observed data and define a distance function


In [ ]:
# Summary statistics of the observed data
data_mu = np.mean(data)
data_var = np.var(data)

# Distance function
def f_dist(x):
    mu_err = np.abs(np.mean(x) - data_mu) / data_mu
    var_err = np.abs(np.var(x) - data_var) / data_var
    return mu_err + var_err

### Define a simulation function

In [ ]:
# Simulation function
def f_sim(particle, n=100):
    return RingModel(particle[0], particle[1], var_const).generate_data(n)

## Run ABC-SMC

In [ ]:
KERNEL_METHOD = 'locm'
NPARTICLES = 1000
NITERS = 5

EPS0 = 5
EPS_PERCENTILE = 0.15

particles, weights, results_dict = abcsmc(
    nparticles=NPARTICLES, 
    nparams=2, 
    prior_list=prior_list, 
    niters=NITERS,
    sim_func=f_sim,
    dist_func=f_dist, 
    eps0=EPS0, 
    eps_percentile=EPS_PERCENTILE, 
    min_eps=0, 
    kernel_method=KERNEL_METHOD
)

particle_history = results_dict['particle_history']
weight_history = results_dict['weight_history']
score_history = results_dict['score_history']
acceptance_rates = results_dict['acceptance_rates']
epsilon_history = results_dict['epsilon_history']


## Plot results

In [ ]:
plot_results(
    particle_history, weight_history, score_history, acceptance_rates,
    epsilon_history, prior_list, 
    pname1=pname1, pname2=pname2,
    save=True, imgdir=imgdir
)